In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

ModuleNotFoundError: No module named 'cv2'

In [4]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N2.xml,1804,2493,1734,1882
2,./images\N3.xml,73,260,381,462
3,./images\N4.xml,1326,1830,1464,1607
4,./images\N5.xml,938,1173,709,762


In [3]:
import xml.etree.ElementTree as xet

## Read data
    -Gettting all filename from xml

In [4]:
filename = df['filepath'][0]
filename

'./images\\N1.xml'

In [5]:
# function to retrive filename from the xmlfile
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images\\N1.jpeg'

In [7]:
#get all filename and store in image_path as a list
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images\\N1.jpeg',
 './images\\N2.jpeg',
 './images\\N3.jpeg',
 './images\\N4.jpeg',
 './images\\N5.jpeg',
 './images\\N6.jpeg',
 './images\\N7.jpeg',
 './images\\N8.jpeg',
 './images\\N9.jpeg',
 './images\\N11.jpeg',
 './images\\N12.jpeg',
 './images\\N14.jpeg',
 './images\\N15.jpeg',
 './images\\N16.jpeg',
 './images\\N17.jpeg',
 './images\\N18.jpeg',
 './images\\N19.jpeg',
 './images\\N20.jpeg',
 './images\\N21.jpeg',
 './images\\N22.jpeg',
 './images\\N23.jpeg',
 './images\\N24.jpeg',
 './images\\N25.jpeg',
 './images\\N27.jpeg',
 './images\\N28.jpeg',
 './images\\N30.jpeg',
 './images\\N31.jpeg',
 './images\\N32.jpeg',
 './images\\N33.jpeg',
 './images\\N34.jpeg',
 './images\\N35.jpeg',
 './images\\N36.jpeg',
 './images\\N37.jpeg',
 './images\\N38.jpeg',
 './images\\N40.jpeg',
 './images\\N42.jpeg',
 './images\\N43.jpeg',
 './images\\N44.jpeg',
 './images\\N45.jpeg',
 './images\\N46.jpeg',
 './images\\N47.jpeg',
 './images\\N48.jpeg',
 './images\\N49.jpeg',
 './images\\N50.jpeg

## verify image and output

In [8]:
file_path = image_path[0]
file_path


'./images\\N1.jpeg'

In [9]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Data Preprocessing

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img , img_to_array

In [12]:
labels = df.iloc[:,1:].values 
#selects all rows and all columns of a  df except for the first column, and returns the values as a NumPy array.

In [13]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[0]
    img_array = cv2.imread(image)
    h,w,d = img_array.shape
    #preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_array = img_to_array(load_image)
    norm_load_image_array = load_image_array/255.0
    #normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w , xmax/w
    nymin,nymax = ymin/h , ymax/h
    labels_norm = (nxmin,nxmax,nymin,nymax)
    
    data.append(norm_load_image_array)
    output.append(labels_norm)


In [14]:
data

[array([[[0.8352941 , 0.9764706 , 0.96862745],
         [0.8392157 , 1.        , 0.99607843],
         [0.8392157 , 1.        , 0.99607843],
         ...,
         [0.5764706 , 0.44313726, 0.4392157 ],
         [0.57254905, 0.4509804 , 0.44313726],
         [0.6431373 , 0.5254902 , 0.5176471 ]],
 
        [[0.9137255 , 1.        , 1.        ],
         [0.88235295, 1.        , 1.        ],
         [0.87058824, 0.99607843, 0.99215686],
         ...,
         [0.58431375, 0.44313726, 0.44313726],
         [0.52156866, 0.3882353 , 0.39215687],
         [0.54901963, 0.41568628, 0.41960785]],
 
        [[0.9019608 , 0.9882353 , 0.9764706 ],
         [0.8980392 , 1.        , 0.99607843],
         [0.88235295, 1.        , 1.        ],
         ...,
         [0.5529412 , 0.39607844, 0.4       ],
         [0.5647059 , 0.4117647 , 0.42352942],
         [0.5411765 , 0.4       , 0.40784314]],
 
        ...,
 
        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 0.99215

In [15]:
x = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [16]:
x.shape , y.shape

((225, 224, 224, 3), (225, 4))

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.8,random_state = 0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

Deep Learning Model

In [18]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [19]:
#CNN model
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False

#--------------------------------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

#defining i/p and o/p and downloading inception_resnetV2 model
model = Model(inputs = inception_resnet.input, outputs = headmodel)


219055592/219055592 [==============================] - 33s 0us/step


 compiling model

In [20]:
model.compile(loss='mse' ,optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 25, 25, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 25, 25, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 25, 25, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 25, 25, 48)   0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_169 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_169[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_167 (Batch  (None, 5, 5, 192)   576         ['conv2d_167[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_175 (Batch  (None, 5, 5, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_178 (Batch  (None, 5, 5, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_175[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_178[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_186[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 custom_scale_layer_35 (CustomS  (None, 5, 5, 2080)  0           ['block8_5_ac[0][0]',            
 caleLayer)                                                       'block8_6_conv[0][0]']          
          

                                                                                                  
 custom_scale_layer_37 (CustomS  (None, 5, 5, 2080)  0           ['block8_7_ac[0][0]',            
 caleLayer)                                                       'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['custom_scale_layer_37[0][0]']  
                                                                                                  
 conv2d_196 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 5, 5, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 500)          19200500    ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 250)          125250      ['dense[0][0]']                  
                                                                                                  
 dense_2 (

model training

In [21]:
from tensorflow.keras.callbacks import TensorBoard

In [22]:
tfb = TensorBoard('object_detection')

In [24]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100, 
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 [==============================] - 54s 3s/step - loss: 0.1148 - val_loss: 0.1006
Epoch 2/100
18/18 [==============================] - 45s 3s/step - loss: 0.1088 - val_loss: 0.0900
Epoch 3/100
18/18 [==============================] - 53s 3s/step - loss: 0.0991 - val_loss: 0.0912
Epoch 4/100
18/18 [==============================] - 52s 3s/step - loss: 0.1040 - val_loss: 0.0959
Epoch 5/100
18/18 [==============================] - 179s 10s/step - loss: 0.0951 - val_loss: 0.0845
Epoch 6/100
18/18 [==============================] - 48s 3s/step - loss: 0.0933 - val_loss: 0.0977
Epoch 7/100
18/18 [==============================] - 46s 3s/step - loss: 0.0988 - val_loss: 0.0796
Epoch 8/100
18/18 [==============================] - 39s 2s/step - loss: 0.0913 - val_loss: 0.0997
Epoch 9/100
18/18 [==============================] - 38s 2s/step - loss: 0.0905 - val_loss: 0.0865
Epoch 10/100
18/18 [==============================] - 39s 2s/step - loss: 0.0938 - val_loss: 0.0759
Epoch 1

Epoch 83/100
18/18 [==============================] - 38s 2s/step - loss: 0.0436 - val_loss: 0.0387
Epoch 84/100
18/18 [==============================] - 38s 2s/step - loss: 0.0440 - val_loss: 0.0422
Epoch 85/100
18/18 [==============================] - 38s 2s/step - loss: 0.0491 - val_loss: 0.0392
Epoch 86/100
18/18 [==============================] - 38s 2s/step - loss: 0.0437 - val_loss: 0.0438
Epoch 87/100
18/18 [==============================] - 38s 2s/step - loss: 0.0438 - val_loss: 0.0400
Epoch 88/100
18/18 [==============================] - 38s 2s/step - loss: 0.0444 - val_loss: 0.0389
Epoch 89/100
18/18 [==============================] - 38s 2s/step - loss: 0.0422 - val_loss: 0.0480
Epoch 90/100
18/18 [==============================] - 38s 2s/step - loss: 0.0462 - val_loss: 0.0402
Epoch 91/100
18/18 [==============================] - 38s 2s/step - loss: 0.0427 - val_loss: 0.0387
Epoch 92/100
18/18 [==============================] - 38s 2s/step - loss: 0.0430 - val_loss: 0.0385


In [25]:
history = model.fit(x=x_train, y=y_train, batch_size=10, 
                   epochs=200 ,validation_data =(x_test,y_test), callbacks=[tfb], initial_epoch = 100)

Epoch 101/200
18/18 [==============================] - 46s 3s/step - loss: 0.0438 - val_loss: 0.0394
Epoch 102/200
18/18 [==============================] - 43s 2s/step - loss: 0.0444 - val_loss: 0.0394
Epoch 103/200
18/18 [==============================] - 37s 2s/step - loss: 0.0431 - val_loss: 0.0395
Epoch 104/200
18/18 [==============================] - 37s 2s/step - loss: 0.0431 - val_loss: 0.0393
Epoch 105/200
18/18 [==============================] - 38s 2s/step - loss: 0.0438 - val_loss: 0.0391
Epoch 106/200
18/18 [==============================] - 41s 2s/step - loss: 0.0431 - val_loss: 0.0394
Epoch 107/200
18/18 [==============================] - 37s 2s/step - loss: 0.0433 - val_loss: 0.0385
Epoch 108/200
18/18 [==============================] - 58s 3s/step - loss: 0.0434 - val_loss: 0.0400
Epoch 109/200
18/18 [==============================] - 45s 3s/step - loss: 0.0434 - val_loss: 0.0388
Epoch 110/200
18/18 [==============================] - 53s 3s/step - loss: 0.0438 - val_los

18/18 [==============================] - 38s 2s/step - loss: 0.0435 - val_loss: 0.0392
Epoch 183/200
18/18 [==============================] - 39s 2s/step - loss: 0.0431 - val_loss: 0.0396
Epoch 184/200
18/18 [==============================] - 39s 2s/step - loss: 0.0429 - val_loss: 0.0391
Epoch 185/200
18/18 [==============================] - 43s 2s/step - loss: 0.0425 - val_loss: 0.0391
Epoch 186/200
18/18 [==============================] - 40s 2s/step - loss: 0.0428 - val_loss: 0.0389
Epoch 187/200
18/18 [==============================] - 47s 3s/step - loss: 0.0435 - val_loss: 0.0390
Epoch 188/200
18/18 [==============================] - 40s 2s/step - loss: 0.0430 - val_loss: 0.0386
Epoch 189/200
18/18 [==============================] - 41s 2s/step - loss: 0.0462 - val_loss: 0.0390
Epoch 190/200
18/18 [==============================] - 37s 2s/step - loss: 0.0433 - val_loss: 0.0408
Epoch 191/200
18/18 [==============================] - 42s 2s/step - loss: 0.0428 - val_loss: 0.0385
Epoc

In [26]:
model.save('./models/object_detection.h5')